In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### RF (car_model을 타겟 인코딩 / 나머지는 그대로)
그런데 파이썬으로 못할거같음... 랜포는 파이썬으로 돌리려면 나머지도 다 인코딩 해야함, 근데 knime으로는 그냥 그대로 사용할 수 있기 때문.. 그래서 일단 파이썬으로는 train/test 분리한 채로 인코딩까지 해놓고 이걸 그대로 knime으로 들고가서 수행해보기! => Qi group2 > dataset > 내에 train과 test 각각 넣어놨음

In [ ]:
=import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

np.random.seed(42)  # 랜덤 시드 고정

# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/QI group2/dataset/Used_cars.csv', encoding="latin1")

# Target Encoding (car_model 처리) - K-Fold 방식으로 데이터 누수 방지
def target_encode(train, test, target_col, n_splits=5):
    """
    K-Fold 기반 Target Encoding을 적용하여 데이터 누수를 방지하는 함수
    train: 훈련 데이터
    test: 테스트 데이터
    target_col: 인코딩할 범주형 컬럼명
    n_splits: KFold 개수 (k=5로 설정)
    """
    train = train.copy().reset_index(drop=True)
    test = test.copy().reset_index(drop=True)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    train[target_col + "_TE"] = np.nan  # 새로운 컬럼 생성

    for train_idx, valid_idx in kf.split(train):
        fold_train, fold_valid = train.iloc[train_idx], train.iloc[valid_idx]
        mean_map = fold_train.groupby(target_col)['listed_price'].mean().to_dict()
        train.loc[valid_idx, target_col + "_TE"] = fold_valid[target_col].map(mean_map)

    # 테스트 데이터에 전체 훈련 데이터에서 계산된 평균 적용
    overall_mean = train.groupby(target_col)['listed_price'].mean().to_dict()
    test[target_col + "_TE"] = test[target_col].map(lambda x: overall_mean.get(x, np.nan))

    return train, test


### 모델 부분 ###

# 8:2로 데이터 분할 (Train-Test Split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Target Encoding 적용
train_df, test_df = target_encode(train_df, test_df, 'car_model')

# 결측값 처리
train_df['car_model_TE'].fillna(train_df['car_model_TE'].mean(), inplace=True)
test_df['car_model_TE'].fillna(train_df['car_model_TE'].mean(), inplace=True)

# car_model 원본 컬럼 삭제 (Target Encoding 완료)
train_df.drop(columns=['car_model'], inplace=True)
test_df.drop(columns=['car_model'], inplace=True)

# CSV 파일 저장 (KNIME에서 불러오기 위함)
train_df.to_csv('/content/drive/MyDrive/QI group2/dataset/Used_cars_train_encoded.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/QI group2/dataset/Used_cars_test_encoded.csv', index=False)

print("Target Encoding 완료. CSV 파일이 저장되었습니다.")

<ipython-input-3-2da4ff57adf9>:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['car_model_TE'].fillna(train_df['car_model_TE'].mean(), inplace=True)
<ipython-input-3-2da4ff57adf9>:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(v

Target Encoding 완료. CSV 파일이 저장되었습니다.


### Knime에서 돌릴 때 주의할 점!!

- train과 test를 절대 섞으면 안됨 (python에서 인코딩 분리했기 때문)

- target encoding이 적용된 csv를 raw data(Used_cars.csv)와 절대 혼동하지 않도록 관리

- 모델 학습 시 독립 변수(x)와 종속 변수(y)를 구분 (당연한 소리)